In [50]:
from mxnet import autograd, nd

num_inputs = 2 # this time the coeffecients consist of a 2*1 vector + bias
num_examples = 1000
params = nd.array([2, -3.4]).reshape(-1, 1)
bias = nd.array([12])
Xmat = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
y = nd.dot(Xmat, params) + bias
y += nd.random.normal(scale=0.01, shape=y.shape)


[12.]
<NDArray 1 @cpu(0)>


In [43]:
from mxnet.gluon import data as gdata

batch_size = 10
dataset = gdata.ArrayDataset(Xmat, y)
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

for X, y in data_iter:
    print(X, y)
    break



[[-1.7489222   0.82156986]
 [-1.3056092   0.28977817]
 [-0.09401369  0.565057  ]
 [ 0.58872265  2.081384  ]
 [-0.44547397  1.4390576 ]
 [ 1.284168    0.67732435]
 [ 0.57521987 -0.13300505]
 [-0.6092996  -1.3266394 ]
 [-2.0753095   0.0612787 ]
 [ 1.203621    1.4827062 ]]
<NDArray 10x2 @cpu(0)> 
[[ 5.7204714]
 [ 8.409523 ]
 [ 9.876771 ]
 [ 6.1004915]
 [ 6.2284875]
 [12.278732 ]
 [13.602124 ]
 [15.291996 ]
 [ 7.6392174]
 [ 9.368144 ]]
<NDArray 10x1 @cpu(0)>


In [44]:
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(1)) #fully connected layer


In [45]:
from mxnet import init

net.initialize(init.Normal(sigma=5))

In [46]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()

In [47]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

In [62]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(X), y)
    print(f'epochs: {epoch},  loss: {l.mean().asnumpy()} ')
    
print(f'true parameters: {nd.concat(params, bias.reshape(1,-1), dim=0)}, learned parameters: {net[0].weight.data()} {net[0].bias.data()}')

epochs: 1,  loss: [9.268084e-05] 
epochs: 2,  loss: [7.08956e-05] 
epochs: 3,  loss: [5.924445e-05] 
(1, 1)
true parameters: 
[[ 2. ]
 [-3.4]
 [12. ]]
<NDArray 3x1 @cpu(0)>, learned parameters: 
[[ 2.0000014 -3.4001296]]
<NDArray 1x2 @cpu(0)> 
[12.000095]
<NDArray 1 @cpu(0)>
